In [1]:
#!/usr/bin/env python
import asyncio
import pyppeteer as ptr
from typing import Optional
import pandas as pd
from glob import glob
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from typing import Callable
import csv

In [2]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/urmzd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/urmzd/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/urmzd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
PosTag = tuple[str, str]
PosTagList = list[PosTag]
StemWord = str
StemWordList = list[StemWord]

In [5]:
def get_name_and_extension(file_path: str) -> tuple[str, str]:
    regex = re.compile(r"(.*)/(.*)\.(.*)")
    return regex.match(file_path).group(2,3)

In [15]:
## Scraper.
async def download_html(browser: ptr.browser.Browser, page: ptr.page.Page, url: str, selector: str) -> Optional[str]:
    await page.goto(url, waitUntil="load", timeout=0)
    content = await page.querySelector(selector)

    html = ''
    if content:
        html = await page.evaluate('(element) => element.textContent', content)
        
    return html

def write_to_resource_target(target_path: str, file_name: str, content: StemWordList, extension="txt") -> None:
    with open(f"{target_path}/{file_name}.{extension}", "w") as file:
        file.write(r"\n".join(content))


async def get_training_data_from_folder(source_path: str, target_path: str, force=False) -> None:
    browser = await ptr.launch(headless=True)
    page = await browser.newPage()
    
    glob_pattern = "/**/*.csv"
    source_files = glob(source_path + glob_pattern, recursive=True)
    target_files = glob(target_path + glob_pattern, recursive=True)
    target_file_names = [get_name_and_extension(file_path)[0] for file_path in target_files]
    
    for file_path in source_files:
        [file_name, file_extension] = get_name_and_extension(file_path)
        print(file_name, file_extension, "FILE")
        
        if not file_name in target_file_names or force:
            result = await get_training_data(browser, page, file_path)
            write_to_resource_target(target_path, file_name, result)        
                
    await browser.close()

async def get_training_data(browser: ptr.browser.Browser, page: ptr.page.Page, file_path: str) -> StemWordList:
    df = pd.read_csv(file_path)
    df.rename(columns=lambda x: x.strip(), inplace=True)

    words = []
    if isinstance(df, pd.DataFrame):
        for _, row in df.iterrows():
            print(row)
            result = await download_html(browser, page, row["link"], row["selector"])
            words.extend(clean_up_words(tokenize(result)))
    
    return words

In [16]:
## Cleaners.
def tokenize(data: str):
    tokenized_words = nltk.word_tokenize(data)
    mutated_words = nltk.pos_tag(tokenized_words)
    return mutated_words

def filter_words(x: PosTag, *fns: Callable[[PosTag], bool]) -> bool:
    if fns:
        if fns[0](x):
            return True
        else:
            return filter_words(x, *fns[1:])
    
    return True
    
def filter_by_punctuation(x: PosTag) -> bool:
    return x[0] != x[1]

def filter_by_stop_word(x: PosTag) -> bool:
    return x[0] not in stopwords.words("english")

def filter_by_alphabet(x: PosTag) -> bool:
    regex = re.compile("^([a-zA-Z]|')+$")
    return regex.match(x[0])

def map_by_stem_words(x: PosTag, ps=PorterStemmer()) -> StemWord:
    return ps.stem(x[0], True)

def map_words(x: PosTag, *fns: Callable[[PosTag], StemWord]) -> StemWord:
    if fns:
        return fns[0](x)
    
    return map_words(x, *fns[1:])
            
def clean_up_words(words: PosTagList) -> StemWordList:
    filtered_words = list(filter(
        lambda x: filter_words(x, filter_by_punctuation, filter_by_stop_word, filter_by_alphabet),
        words
        )
    )
    return map(lambda x: map_words(x, map_by_stem_words), filtered_words)

In [17]:
# type: ignore
await (get_training_data_from_folder("../resources/sources", "../resources/targets", force=True))

depression csv FILE
selector                                           #topicText
link        https://www.uptodate.com/contents/depression-t...
Name: 0, dtype: object
selector                                        #main-content
link        https://www.mayoclinic.org/diseases-conditions...
Name: 1, dtype: object
selector                                     #mw-content-text
link        https://simple.wikipedia.org/wiki/Depression_(...
Name: 2, dtype: object
selector                                         #maincontent
link        https://www.nhs.uk/mental-health/conditions/cl...
Name: 3, dtype: object
selector    #__next > div.css-fdjy12 > div:nth-child(5) > ...
link        https://www.healthline.com/health/depression#s...
Name: 4, dtype: object
selector    #__next > div.css-fdjy12 > div:nth-child(5) > ...
link        https://www.medicalnewstoday.com/articles/3213...
Name: 5, dtype: object
selector                   #skip > div:nth-child(1) > article
link        https://www.hopkinsmedic

In [20]:
# Parsers
def get_cleaned_data_from_file(input_file_path: str) -> Optional[list[str]]:
    with open(input_file_path, "r") as file:
        return list(file.readlines())

def get_cleaned_data_from_folder(input_path: str) -> dict[str, PosTagList]:
    input_files = glob(input_path + "/**/*", recursive=True)
    
    data = dict()
    for file_path in input_files:
        [file_name, file_extension] = get_name_and_extension(file_path)
        classification_data = get_cleaned_data_from_file(file_path)
        data[file_name] = classification_data
        
    return data


def group_by_tags(pos_tag_list: PosTagList) -> dict[str, list[str]]:
    groups = dict()
    
    for value,tag in pos_tag_list:
        if tag in groups:
            groups[tag]
            groups[tag].append(value)
        else:
            groups[tag] = [value]
            
    return groups
    

result = get_cleaned_data_from_folder("../resources/targets")
result

{'migraine': ["introduct\\n—\\nmigrain\\nis\\na\\ncommon\\nepisod\\ndisord\\n,\\nthe\\nhallmark\\nof\\nwhich\\nis\\na\\ndisabl\\nheadach\\ngener\\nassoci\\nwith\\nnausea\\nand/or\\nlight\\nand\\nsound\\nsensit\\n.\\nthe\\nacut\\ntreatment\\nof\\nmigrain\\nin\\nadult\\nis\\nreview\\nhere\\n.\\nprevent\\ntreatment\\nof\\nmigrain\\nin\\nadult\\nis\\ndiscuss\\nsepar\\n.\\n(\\nsee\\n``\\nprevent\\ntreatment\\nof\\nepisod\\nmigrain\\nin\\nadult\\n''\\n.\\n)\\nthe\\npathophysiolog\\n,\\nclinic\\nmanifest\\n,\\nand\\ndiagnosi\\nof\\nthe\\nmigrain\\nare\\nalso\\ndiscuss\\nsepar\\n.\\n(\\nsee\\n``\\npathophysiolog\\n,\\nclinic\\nmanifest\\n,\\nand\\ndiagnosi\\nof\\nmigrain\\nin\\nadult\\n''\\n.\\n)\\napproach\\nto\\ntreatment\\n—\\nthe\\nabort\\n(\\nsymptomat\\n)\\ntherapi\\nof\\nmigrain\\nrang\\nfrom\\nthe\\nuse\\nof\\nsimpl\\nanalges\\nsuch\\nas\\nnonsteroid\\nanti-inflammatori\\ndrug\\n(\\nnsaid\\n)\\nor\\nacetaminophen\\nto\\ntriptan\\n,\\nantiemet\\n,\\ncalcitonin\\ngene-rel\\npeptid\\n(\\n

In [ ]:
print (type(get_cleaned_data_from_folder("../resources/targets")))